<h1>
<center>
Midterm 1
</center>
</h1>
<div class=h1_cell>

You will be working with a new table. It contains data from movies.
  <p>
Note I am trying something new to help you check your answers against mine. I am putting my answers into the text cell that introduces the question. Once you get your answer you can scroll up to see if matches my answer.
</div>

In [1]:
import pandas as pd
import os

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTST3CURFkC0lHSfR9mGzBAvCmLWl06kw9NSBC3UuD9hA1SlPjh807BRxfnvhq5hrFyRz5ZCTGrs7fr/pub?output=csv'
movie_table = pd.read_csv(url)
len(movie_table)

5043

In [2]:
!rm library_w19_week4b.py

rm: cannot remove 'library_w19_week4b.py': No such file or directory


In [3]:
#load the lbirary from week 4
from google.colab import files
files.upload()

Saving library_w19_week4b.py to library_w19_week4b.py


{'library_w19_week4b.py': b'def predictor_case(row, pred, target):\n  case_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\n  actual = row[target]\n  prediction = row[pred]\n  case = case_dict[(prediction, actual)]\n  return case\n\ndef accuracy(cases):\n    tp = cases[\'true_positive\']\n    tn = cases[\'true_negative\']\n    fp = cases[\'false_positive\']\n    fn = cases[\'false_negative\']\n    return (tp + tn)/(tp+tn+fp+fn)\n\ndef f1(cases):\n    \n    #the heart of the matrix\n    tp = cases[\'true_positive\']\n    fn = cases[\'false_negative\']\n    tn = cases[\'true_negative\']\n    fp = cases[\'false_positive\']\n    \n    #other measures we can derive\n    recall = 1.0*tp/(tp+fn)  # positive correct divided by total positive in the table\n    precision = 1.0*tp/(tp+fp) # positive correct divided by all positive predictions made\n    \n    #now for the one we want\n    f1 = 2/(1/recall + 1/precision)\n    \n    r

In [4]:
from library_w19_week4b import *
%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 f1	 find_best_splitter	 generate_table	 gig	 gini	 
informedness	 path_id	 predictor_case	 probabilities	 reorder_paths	 tree_predictor	 


<div class=h1_cell>

Roughly 5000 rows x 29 columns.

</div>

<div class=h1_cell>

Let's drop columns we will not be using in this practice midterm.

</div>

In [0]:
drop_list = [
     'director_name',
     'num_critic_for_reviews',
     'duration',
     'director_facebook_likes',
     'actor_3_facebook_likes',
     'actor_2_name',
     'actor_1_facebook_likes',
     'gross',
     'actor_1_name',
     'movie_title',
     'num_voted_users',
     'cast_total_facebook_likes',
     'actor_3_name',
     'facenumber_in_poster',
     'plot_keywords',
     'movie_imdb_link',
     'num_user_for_reviews',
     'language',
     'content_rating',
     'title_year',
     'actor_2_facebook_likes',
     'aspect_ratio',
     'movie_facebook_likes'
]

In [6]:
movie_table = movie_table.drop(drop_list, axis=1)
movie_table.head()

,color,genres,country,budget,imdb_score
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8
3,Color,Action|Thriller,USA,250000000.0,8.5
4,NaN,Documentary,NaN,NaN,7.1


In [0]:
pd.set_option('display.max_columns', None)


In [8]:
movie_table.describe(include='all')

,color,genres,country,budget,imdb_score
count,5024,5043,5038,4.551000e+03,5043.000000
unique,2,914,65,NaN,NaN
top,Color,Drama,USA,NaN,NaN
freq,4815,236,3807,NaN,NaN
mean,NaN,NaN,NaN,3.975262e+07,6.442138
std,NaN,NaN,NaN,2.061149e+08,1.125116
min,NaN,NaN,NaN,2.180000e+02,1.600000
25%,NaN,NaN,NaN,6.000000e+06,5.800000
50%,NaN,NaN,NaN,2.000000e+07,6.600000
75%,NaN,NaN,NaN,4.500000e+07,7.200000


<hr>
<h1>
1: Produce new target column (10 points)
</h1>
<p>
<div class=h1_cell>
<p>
I'd like a new binary column that will act as our target. Call the new column `winner`. Set its value as follows:
if `imdb_score >= 7.5` the value is `1` else `0`.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=RIGHT>
<img src='https://www.dropbox.com/s/zuqw01czvet00jv/q1.png?raw=1'>
  </div>
<hr>

In [0]:
#Your code here
movie_table['winner'] = movie_table.apply(lambda row: 1 if row['imdb_score'] >= 7.5 else 0, axis = 1)

In [10]:
movie_table.head()  #check your answer against above

,color,genres,country,budget,imdb_score,winner
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9,1
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1,0
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8,0
3,Color,Action|Thriller,USA,250000000.0,8.5,1
4,NaN,Documentary,NaN,NaN,7.1,0


<hr>
<h1>
2: Bin `budget` then one hot encode (20 points)
</h1>
<p>
<div class=h1_cell>
<p>
Use bins ['low', 'medium', 'high'].
<p>
  <b>Important</b>: produce equal sized bins in terms of rows in those bins. `qcut` will do this for you.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=RIGHT>
<img src='https://www.dropbox.com/s/vr78cq4fo9z2tem/q2.png?raw=1'>
  </div>
<hr>

In [0]:
#your binning code here - remember the qcut
bins = 3
bin_names = ['low', 'medium','high']
movie_table['budget_bin'] = pd.qcut(movie_table['budget'], bins, labels = bin_names)

In [13]:
movie_table.head()  #check against above budget_bin column

,color,genres,country,budget,imdb_score,winner,budget_bin
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9,1,high
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1,0,high
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8,0,high
3,Color,Action|Thriller,USA,250000000.0,8.5,1,high
4,NaN,Documentary,NaN,NaN,7.1,0,NaN


Get value counts and compare with mine:
<pre>
low       1592
high      1508
medium    1451
</pre>

In [14]:
#compute value counts and compare with above
movie_table['budget_bin'].value_counts()

low       1592
high      1508
medium    1451
Name: budget_bin, dtype: int64

Now ohe the `budget_bin` column using `budget` as the prefix. Check against above.

In [0]:
#your code here
one_hot_budget = pd.get_dummies(movie_table['budget_bin'], prefix='budget',dummy_na=True)
movie_table = movie_table.join(one_hot_budget)

In [16]:
movie_table.head()  #check against above

,color,genres,country,budget,imdb_score,winner,budget_bin,budget_low,budget_medium,budget_high,budget_nan
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9,1,high,0,0,1,0
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1,0,high,0,0,1,0
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8,0,high,0,0,1,0
3,Color,Action|Thriller,USA,250000000.0,8.5,1,high,0,0,1,0
4,NaN,Documentary,NaN,NaN,7.1,0,NaN,0,0,0,1


<hr>
<h1>
3: Create a new column for thrillers (5 points)
</h1>
<p>
<div class=h1_cell>
<p>
Create a new colum `thriller`. Its value is 1 if the `genres` column contains the word Thriller else 0.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=RIGHT>
<img src='https://www.dropbox.com/s/if2vpz0o1pa9bo6/q3.png?raw=1'>
  </div>
<hr>


In [0]:
#your code goes here
movie_table['thriller'] = movie_table.apply(lambda row: 1 if 'Thriller' in row['genres'] else 0, axis = 1)


In [18]:
movie_table.head()  #check against my answer

,color,genres,country,budget,imdb_score,winner,budget_bin,budget_low,budget_medium,budget_high,budget_nan,thriller
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9,1,high,0,0,1,0,0
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1,0,high,0,0,1,0,0
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8,0,high,0,0,1,0,1
3,Color,Action|Thriller,USA,250000000.0,8.5,1,high,0,0,1,0,1
4,NaN,Documentary,NaN,NaN,7.1,0,NaN,0,0,0,1,0


<hr>
<h1>
4: New column usa_made (5 points)
</h1>
<p>
<div class=h1_cell>
<p>
Create a new column `usa_made` that has 1 if the `country` column is USA and 0 otherwise.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=RIGHT>
<img src='https://www.dropbox.com/s/jhhlwqbtkzoy09l/q4.png?raw=1'>
  </div>
<hr>


In [0]:
#your code goes here
movie_table['usa_made'] = movie_table.apply(lambda row: 1 if row['country'] == 'USA' else 0, axis=1)


In [20]:
movie_table.head() #check against my answer

,color,genres,country,budget,imdb_score,winner,budget_bin,budget_low,budget_medium,budget_high,budget_nan,thriller,usa_made
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9,1,high,0,0,1,0,0,1
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1,0,high,0,0,1,0,0,1
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8,0,high,0,0,1,0,1,0
3,Color,Action|Thriller,USA,250000000.0,8.5,1,high,0,0,1,0,1,1
4,NaN,Documentary,NaN,NaN,7.1,0,NaN,0,0,0,1,0,0


<hr>
<h1>
5: OHE `color` (10 points)
</h1>
<p>
<div class=h1_cell>
<p>
ohe the column `color` using `film_type` as the prefix.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=RIGHT>
<img src='https://www.dropbox.com/s/td5k13eiwg4yw11/q5.png?raw=1'>
  </div>
<hr>

In [0]:
#your code here
one_hot_color = pd.get_dummies(movie_table['color'], prefix='film_type', dummy_na = True)
movie_table = movie_table.join(one_hot_color)



In [23]:
movie_table.head()  #check against above (only shows the last 3 columns)

,color,genres,country,budget,imdb_score,winner,budget_bin,budget_low,budget_medium,budget_high,budget_nan,thriller,usa_made,film_type_Black and White,film_type_Color,film_type_nan
0,Color,Action|Adventure|Fantasy|Sci-Fi,USA,237000000.0,7.9,1,high,0,0,1,0,0,1,0,1,0
1,Color,Action|Adventure|Fantasy,USA,300000000.0,7.1,0,high,0,0,1,0,0,1,0,1,0
2,Color,Action|Adventure|Thriller,UK,245000000.0,6.8,0,high,0,0,1,0,1,0,0,1,0
3,Color,Action|Thriller,USA,250000000.0,8.5,1,high,0,0,1,0,1,1,0,1,0
4,NaN,Documentary,NaN,NaN,7.1,0,NaN,0,0,0,1,0,0,0,0,1


<hr>
<h1>
6: Build a tree with depth 3 and default gig-cutoff (5 points)
</h1>
<p>
<div class=h1_cell>
<p>
Call your tree `tree1`. I'll give you the splitter column candidates to use below. Use `winner` as the target column.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=RIGHT>
<img src='https://www.dropbox.com/s/ap24g3bpau2i10z/q6.png?raw=1'>
  </div>
<hr>



In [0]:
column_candidates = [
        'thriller',
        'budget_low', 'budget_medium', 'budget_high', 'budget_nan',
        'usa_made',
        'film_type_Color', 'film_type_Black and White', 'film_type_nan'
]

In [0]:
#your code here
tree1 = build_tree_iter(movie_table, column_candidates, 'winner', {'max-depth': 3})

In [26]:
print(len(tree1['paths']))  #check against my answer
tree1['paths']

8


[{'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_0', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_0',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('usa_made_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.019071756174530274,
  'predic

<hr>
<h1>
7: Compute scores for tree1 (5 points)
</h1>
<p>
<div class=h1_cell>
<p>
Print accuracy, f1 and informedness scores.
<p>
</div>


<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=left>
<pre>
true_negative     4104
false_negative     827
true_positive       60
false_positive      52
(0.825698988697204, 0.12012012012012013, 0.055131712154931733)
</pre>
  </div>
<hr>

In [0]:
#your code here
movie_results = movie_table[['winner']].copy()
movie_results['tree_1'] = movie_table.apply(lambda row: tree_predictor(row, tree1), axis= 1)
movie_results['tree_1_type'] = movie_results.apply(lambda row: predictor_case(row, pred='tree_1', target='winner'), axis=1)
tree_1_types = movie_results['tree_1_type'].value_counts()

In [33]:
tree_1_types  #check against above

true_negative     4104
false_negative     827
true_positive       60
false_positive      52
Name: tree_1_type, dtype: int64

In [34]:
print((accuracy(tree_1_types), f1(tree_1_types), informedness(tree_1_types))) # (0.825698988697204, 0.12012012012012013, 0.055131712154931733)

(0.825698988697204, 0.12012012012012013, 0.055131712154931733)


<hr>
<h1>
8. Do the same for tree2 (10 points)
</h1>
<p>
<div class=h1_cell>
<p>
Build a tree with depth of 4 and then print accuracy, f1 and informedness scores.
<p>
</div>

<hr>
<center><h3>My output to use as target</h3></center>
<DIV ALIGN=left>
<pre>
true_negative     4108
false_negative     826
true_positive       61
false_positive      48
(0.8266904620265715, 0.12248995983935741, 0.05722157177843146)
</pre>
  </div>
<hr>

In [0]:
#your code here for building tree2
tree2 = build_tree_iter(movie_table, column_candidates, 'winner', {'max-depth': 4})


In [36]:
print(len(tree2['paths']))  #should be 16
#tree2['paths']

16


In [0]:
#your code here for generating scores
movie_results['tree_2'] = movie_table.apply(lambda row: tree_predictor(row, tree2), axis= 1)
movie_results['tree_2_type'] = movie_results.apply(lambda row: predictor_case(row, pred='tree_2', target='winner'), axis=1)
tree_2_types = movie_results['tree_2_type'].value_counts()


In [38]:
tree_2_types  #check against above

true_negative     4108
false_negative     826
true_positive       61
false_positive      48
Name: tree_2_type, dtype: int64

In [39]:
print((accuracy(tree_2_types), f1(tree_2_types), informedness(tree_2_types))) #(0.8266904620265715, 0.12248995983935741, 0.05722157177843146)

(0.8266904620265715, 0.12248995983935741, 0.05722157177843146)


<hr>
<h1>
9: Write a function to find out how many times a column is used in a tree (30 points)
</h1>
<p>
<div class=h1_cell>
<p>
For a specific tree, which columns actually get used when building the tree? And how many times do they appear? Write a function `column_counts` that takes a tree in as input and produces a dictionary of columns that the tree uses as keys and how often as values. Test it on tree1 and tree2.
  <p>
    Note that this has nothing to do with a table. It is just an analysis of the structure of the tree.
<p>
Here are my results for tree1:
<p>
<pre>
{'budget_low': 4,
 'budget_nan': 2,
 'film_type_Black and White': 8,
 'thriller': 4,
 'usa_made': 6}
</pre>
<p>
</div>

In [0]:
#define function here

def column_counts(tree):
  assert (len(tree['paths']) > 0), "Tree must have at least one path"
  columns = {}
  for path in tree['paths']: 
    conjunct = path['conjunction'] #pull out the conjunction 
    cols = map(lambda tuple: tuple[0], conjunct) # get a list of all the columns
    for column in cols:  
      column = column[:-2] # remove the _X
      if columns.get(column) == None: # if column not in the dict already set value to 1
        columns[column] = 1
      else: # else increment the value
        columns[column]+=1
  return columns
    


In [47]:
column_counts(tree1)  #check results against above

{'budget_low': 4,
 'budget_nan': 2,
 'film_type_Black and White': 8,
 'thriller': 4,
 'usa_made': 6}

<div class=h1_cell>

Here are my results for tree2:
<p>
<pre>
{'budget_high': 4,
 'budget_low': 8,
 'budget_nan': 6,
 'film_type_Black and White': 16,
 'thriller': 14,
 'usa_made': 16}
</pre>
<p>
</div>

In [48]:
column_counts(tree2)  #check against above

{'budget_high': 4,
 'budget_low': 8,
 'budget_nan': 6,
 'film_type_Black and White': 16,
 'thriller': 14,
 'usa_made': 16}

<hr>
<h1>
Extra credit: Find shortest path in tree
</h1>
<p>
<div class=h1_cell>
<p>
Write a function to find the shortest paths in a tree. If there is a unique shortest path then your list will just have one path. If there are ties, the list will include all that tied.
<p>
</div>

In [0]:
#your function here
def list_of_shortest(tree):
  shortest = []
  slength = 0
  for path in tree['paths']:
    conjunct = path['conjunction'] #pull out the conjunction 
    if slength == 0: 
      slength = len(conjunct)
      shortest.append(path)
    else: 
      if len(conjunct) <= slength: 
        if len(conjunct) < slength:
          shortest.clear()
        shortest.append(path)
  return shortest

<hr>
<h1>
Here is some testing code
</h1>
<p>
<div class=h1_cell>
<p>
I am making a copy of tree1 then manipulating the first path, i.e., I am slicing it. I can slice it down to one predicate, 2 predicates, etc.
<p>
You can change the value of n from 1 to longest path. For each value of n run your function to make sure you get correct result.
<p>
</div>

In [0]:
import copy

def path_tester(tree, n):
  tree_test = copy.deepcopy(tree)
  slice = tree_test['paths'][0:n]  #slice
  tree_test['paths'] = slice
  print(('length of paths considered', len(slice)))
  return list_of_shortest(tree_test)

In [55]:
path_tester(tree1, 1)  #list of one path of length 3

('length of paths considered', 1)


[{'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1}]

In [56]:
path_tester(tree1, 2)  #list of 2 paths of length 3

('length of paths considered', 2)


[{'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_0', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1}]

In [57]:
path_tester(tree1, 3)  #list of 3 paths of length 3

('length of paths considered', 3)


[{'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_0', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_0',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('usa_made_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.019071756174530274,
  'predic

In [58]:
path_tester(tree1, len(tree1['paths']))  #list of 8 paths of length 3

('length of paths considered', 8)


[{'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('thriller_0', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.005303667858896077,
  'prediction': 1},
 {'conjunction': [('film_type_Black and White_1',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('budget_low_0',
    <function library_w19_week4b.build_pred.<locals>.<lambda>>),
   ('usa_made_1', <function library_w19_week4b.build_pred.<locals>.<lambda>>)],
  'gig_score': 0.019071756174530274,
  'predic